In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torch
!pip3 install deeppavlov
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 6.6 MB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 880 kB 5.3 MB/s 
     |████████████████████████████████| 654 kB 32.9 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 2.9 MB 31.4 MB/s 
     |████████████████████████████████| 3.8 MB 35.7 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 20.1 MB 53.4 MB/s 
     |████████████████████████████████| 6.7 MB 19.7 MB/s 
     |████████████████████████████████| 510 kB 49.3 MB/s 
     |████████████████████████████████| 10.4 M

In [ ]:
!cp  drive/MyDrive/НИР/dataset_roberta.py .
!cp  drive/MyDrive/НИР/functions_for_roberta.py .
from dataset_roberta import CustomDataset
from functions_for_roberta import Classifier

In [ ]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'


In [ ]:
from transformers import AutoModel,  AutoConfig, AutoTokenizer

config = AutoConfig.from_pretrained("blinoff/roberta-base-russian-v0")
tokenizer = AutoTokenizer.from_pretrained("blinoff/roberta-base-russian-v0")
model = AutoModel.from_pretrained("blinoff/roberta-base-russian-v0")
model_ef = AutoModel.from_pretrained("blinoff/roberta-base-russian-v0")

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477M [00:00<?, ?B/s]

Some weights of the model checkpoint at blinoff/roberta-base-russian-v0 were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at blinoff/roberta-base-russian-v0 were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.

In [ ]:
import torch

out_features = model.encoder.layer[1].output.dense.out_features
#model.classifier = torch.nn.Linear(out_features, 2)

In [ ]:
classifier_RoB = Classifier(
    model,
    tokenizer,
    out_features,
    6
)

classifier_RoB_ef = Classifier(
    model_ef,
    tokenizer,
    out_features,
    6
)

In [ ]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path='/content/drive/MyDrive/НИР',
    train='Join_dataset.csv',
    x = 'text',
    y = 'news'
)

# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid', 'test'],   # fields to which the fiald above will be splitted
    split_proportions=[0.5, 0.1, 0.4],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset
    seed=42)  # seed for iteration over dataset

2022-04-15 13:52:22.374 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/valid.csv file
2022-04-15 13:52:22.378 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/test.csv file
2022-04-15 13:52:22.382 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid', 'test']>>


In [ ]:
x_train, y_train = train_iterator.get_instances(data_type='train')

x_valid, y_valid = train_iterator.get_instances(data_type='valid')

x_test, y_test = train_iterator.get_instances(data_type='test')


In [ ]:
classifier_RoB.preparation(x_train, y_train, x_valid, y_valid )
classifier_RoB_ef.preparation(x_train, y_train, x_valid, y_valid )


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
"""attention = classifier_RoB_ef.train_loader.dataset.__getitem__(1)['attention_mask'].to(device)
ids = attention = classifier_RoB_ef.train_loader.dataset.__getitem__(1)['input_ids'].to(device)

out = classifier_RoB_ef.model(
      input_ids=ids.unsqueeze(0),
      attention_mask=attention.unsqueeze(0)
  )
  #out
logits = classifier_RoB_ef.model.classifier(out.pooler_output)
torch.argmax(logits,dim=-1).cpu().numpy()[0]"""


"attention = classifier_RoB_ef.train_loader.dataset.__getitem__(1)['attention_mask'].to(device)\nids = attention = classifier_RoB_ef.train_loader.dataset.__getitem__(1)['input_ids'].to(device)\n\nout = classifier_RoB_ef.model(\n      input_ids=ids.unsqueeze(0),\n      attention_mask=attention.unsqueeze(0)\n  )\n  #out\nlogits = classifier_RoB_ef.model.classifier(out.pooler_output)\ntorch.argmax(logits,dim=-1).cpu().numpy()[0]"

In [ ]:
classifier_RoB_ef.predict(x_test[105])

0

In [ ]:
 torch.cuda.empty_cache()

In [ ]:
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

valid_acc_ep, valid_loss_ep = classifier_RoB_ef.eval()
#predictions_wft = classifier_RoB_ef.predict(x_test) 
predictions = [str(classifier_RoB_ef.predict(t)) for t in x_test]


In [ ]:
valid_acc_ep

tensor(0.6170, device='cuda:0', dtype=torch.float64)

In [ ]:
all_valid_loss, all_valid_acc, all_train_loss, all_train_acc = classifier_RoB.train('RoBerta.pth')

Epoch 1/6
Train loss 0.09347121855493169 accuracy 0.9794
Val loss 0.06910754978974 accuracy 0.991
----------
Epoch 2/6
Train loss 0.046473840621928685 accuracy 0.993
Val loss 0.033041793536162004 accuracy 0.993
----------
Epoch 3/6
Train loss 0.026169456323381746 accuracy 0.9962000000000001
Val loss 0.021821479868842288 accuracy 0.997
----------
Epoch 4/6
Train loss 0.012960929888102692 accuracy 0.9984000000000001
Val loss 0.03809675873797096 accuracy 0.995
----------
Epoch 5/6
Train loss 0.011358947477018228 accuracy 0.9986
Val loss 0.03245405121234944 accuracy 0.996
----------
Epoch 6/6
Train loss 0.006448599323727831 accuracy 0.999
Val loss 0.03215481567129609 accuracy 0.996
----------


In [ ]:
#predictions_ft = [str(classifier_RoB.predict(t)) for t in x_test]
#precision, recall, f1score = precision_recall_fscore_support(y_test, predictions_ft)[:3]

#print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')
precision_, recall_, f1score_ = precision_recall_fscore_support(y_test, predictions)[:3]
print(f'precision: {precision_}, recall: {recall_}, f1score: {f1score_}')

precision: [0.60212672 0.62076879], recall: [0.67277228 0.54646465], f1score: [0.63549217 0.58125168]


In [ ]:
valid_loss_ep

0.6827561004161835

In [ ]:
!pip install plotly

In [ ]:
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

print('Originaly loss of the model:', valid_loss_ep)

fig = go.Figure(
    data=[go.Line(y = all_train_loss, x=[1,2,3,4,5,6])]
)
fig.update_layout(title='Train Loss DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_loss, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Loss DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig.show(renderer="colab")
fig_2.show(renderer="colab")


Originaly loss of the model: 0.6827561004161835


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning: plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.

  DeprecationWarning,
/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
all_train_acc_pre = []
all_valid_acc_pre = []
valid_acc_pre = []

for i in range(6):
  all_train_acc_pre.append(all_train_acc[i].cpu())
  all_valid_acc_pre.append(all_valid_acc[i].cpu())

all_train_acc_pre = np.array(all_train_acc_pre)
all_valid_acc_pre = np.array(all_valid_acc_pre)


In [ ]:
print('Originaly accuracy of the model:', valid_acc_ep)

fig = go.Figure(
    data=[go.Line(y = all_train_acc_pre, x=[1,2,3,4,5,6])]
)

fig.update_layout(title='Train Accuracy DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_acc_pre, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Accuracy DistilRuBERT',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig.show(renderer="colab")
fig_2.show(renderer="colab")


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




Originaly loss of the model: tensor(0.6170, device='cuda:0', dtype=torch.float64)
